In [83]:
import json 
import time
import math
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import datetime, timezone
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import models

from database_crud import CRUD
db = CRUD()

In [ ]:
from bs4 import BeautifulSoup
import requests


url1 = "https://dhlottery.co.kr/gameResult.do?method=byWin&drwNo=" 
lotto = open("로또 번호2.txt", "a", encoding="UTF-8")


for url2 in range(900, 924) :
    url = url1 + str(url2)
    lotto.write(str(url2) + "회 : ")
    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'html.parser')
    data1 = soup.find('div', {'class': 'win_result'})
       
    i = 0
    d = []
    for i in range(0,7) :
        data2 = data1.findAll('span')[i].text  # list에 넣어서 결과 보기
        d.append(data2)
        i = i + 1
    lotto.write(str(d))    
    lotto.write("\n")
      
    
lotto.close()

In [ ]:
s = f"2003-01-11 00:00:00" 
datetime.strptime(s, '%Y-%m-%d %H:%M:%S')

In [ ]:
import json 
import time
import requests
import pandas as pd 
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import datetime, timezone

from database_crud import CRUD
db = CRUD()

def getLottoWinInfo(minDrwNo, maxDrwNo): 
    drwtNo1 = [] 
    drwtNo2 = [] 
    drwtNo3 = [] 
    drwtNo4 = [] 
    drwtNo5 = [] 
    drwtNo6 = []
    bnusNo = [] 
    totSellamnt = [] 
    drwNoDate = [] 
    firstAccumamnt = [] 
    firstPrzwnerCo = [] 
    firstWinamnt = [] 
    
    for i in tqdm(range(minDrwNo, maxDrwNo+1, 1)):  # tqdm : 시간의 경과에 따라 진행률 보여줌
        req_url = "https://dhlottery.co.kr/gameResult.do?method=byWin&drwNo=" + str(i)
        req_html = requests.get(req_url)
        req_soup = BeautifulSoup(req_html.text, 'html.parser')
        req_data = req_soup.find('div', {'class': 'win_result'})
        lottoTimes = req_data.findAll('h4')[0].text.split("회")[0]

        req_api_url = "http://www.dhlottery.co.kr/common.do?method=getLottoNumber&drwNo=" + str(i) 
        req_api_lotto = requests.get(req_api_url)
        lottoNo = req_api_lotto.json() 
        drwtNo1 = lottoNo['drwtNo1']
        drwtNo2 = lottoNo['drwtNo2']
        drwtNo3 = lottoNo['drwtNo3']
        drwtNo4 = lottoNo['drwtNo4']
        drwtNo5 = lottoNo['drwtNo5']
        drwtNo6 = lottoNo['drwtNo6']
        bnusNo = lottoNo['bnusNo']
        totSellamnt = lottoNo['totSellamnt']
        drwNoDate = datetime.strptime(f"{lottoNo['drwNoDate']} 00:00:00", '%Y-%m-%d %H:%M:%S')
        firstAccumamnt = lottoNo['firstAccumamnt']
        firstPrzwnerCo = lottoNo['firstPrzwnerCo']
        # firstWinamnt = lottoNo['firstWinamnt']

        # lotto_dict = {
        #     "lotto_times":lottoTimes,
        #     "draw_time":drwNoDate, 
        #     "num1":drwtNo1, 
        #     "num2":drwtNo2, 
        #     "num3":drwtNo3, 
        #     "num4":drwtNo4, 
        #     "num5":drwtNo5, 
        #     "num6":drwtNo6, 
        #     "bonus":bnusNo,
        #     "sales_amount":totSellamnt, 
        #     "winning_amount":firstAccumamnt, 
        #     "winner_count":firstPrzwnerCo, 
        #     # "1등수령액":firstWinamnt,
        #   } 
        
        now =datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        # timestamp = time.mktime(now.timetuple())
        # print(now)
        db.insertDB(
            schema='public',
            table='LottoHistory',
            colum='lotto_times, num1, num2, num3, num4, num5, num6, bonus, draw_time, create_time, winning_amount, winner_count, sales_amount',
            data=f'{lottoTimes}, {drwtNo1}, {drwtNo2}, {drwtNo3}, {drwtNo4}, {drwtNo5}, {drwtNo6}, {bnusNo}, TIMESTAMP \'{drwNoDate}\', TIMESTAMP \'{now}\', {firstAccumamnt}, {firstPrzwnerCo}, {totSellamnt}'
        )

    # df_lotto = pd.DataFrame(lotto_dict) 
    # return df_lotto

In [84]:
data = db.readDB(schema='public',table='LottoHistory',colum='lotto_times, num1, num2, num3, num4, num5, num6, bonus, draw_time, create_time, winning_amount, winner_count, sales_amount')
dataframe_lotto = pd.DataFrame(data, columns=['lotto_times', 'num1', 'num2', 'num3', 'num4', 'num5', 'num6', 'bonus', 'draw_time', 'create_time', 'winning_amount', 'winner_count', 'sales_amount'])
dataframe_lotto

,lotto_times,num1,num2,num3,num4,num5,num6,bonus,draw_time,create_time,winning_amount,winner_count,sales_amount
0,1,10,23,29,33,37,40,16,2002-12-07,2023-05-14 21:26:33,863604600,0,3681782000
1,2,9,13,21,25,32,42,2,2002-12-14,2023-05-14 21:26:33,0,1,4904274000
2,3,11,16,19,21,27,31,30,2002-12-21,2023-05-14 21:26:34,0,1,4729342000
3,4,14,27,30,31,40,42,2,2002-12-28,2023-05-14 21:26:34,1267147200,0,5271464000
4,5,16,24,29,40,41,42,3,2003-01-04,2023-05-14 21:26:34,3041094900,0,6277102000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1062,1063,3,6,22,23,24,38,30,2023-04-15,2023-05-14 21:40:07,26392183125,7,108347135000
1063,1064,3,6,9,18,22,35,14,2023-04-22,2023-05-14 21:40:08,25615194014,19,109427584000
1064,1065,3,18,19,23,32,45,24,2023-04-29,2023-05-14 21:40:09,25936315132,14,108181727000
1065,1066,6,11,16,19,21,32,45,2023-05-06,2023-05-14 21:40:10,25064208750,15,106440950000


In [85]:
dataframe_actual = dataframe_lotto.loc[:,['num1', 'num2', 'num3', 'num4', 'num5', 'num6', 'bonus', 'winning_amount', 'winner_count', 'sales_amount']]
dataframe_actual

,num1,num2,num3,num4,num5,num6,bonus,winning_amount,winner_count,sales_amount
0,10,23,29,33,37,40,16,863604600,0,3681782000
1,9,13,21,25,32,42,2,0,1,4904274000
2,11,16,19,21,27,31,30,0,1,4729342000
3,14,27,30,31,40,42,2,1267147200,0,5271464000
4,16,24,29,40,41,42,3,3041094900,0,6277102000
...,...,...,...,...,...,...,...,...,...,...
1062,3,6,22,23,24,38,30,26392183125,7,108347135000
1063,3,6,9,18,22,35,14,25615194014,19,109427584000
1064,3,18,19,23,32,45,24,25936315132,14,108181727000
1065,6,11,16,19,21,32,45,25064208750,15,106440950000


In [86]:
len(dataframe_actual)

1067

In [87]:
dataframe_actual.loc[:, ['num1', 'num2', 'num3', 'num4', 'num5', 'num6']].to_numpy(int)

array([[10, 23, 29, 33, 37, 40],
       [ 9, 13, 21, 25, 32, 42],
       [11, 16, 19, 21, 27, 31],
       ...,
       [ 3, 18, 19, 23, 32, 45],
       [ 6, 11, 16, 19, 21, 32],
       [ 7, 10, 19, 23, 28, 33]])

In [88]:
row_count = len(dataframe_actual)

# 당첨번호를 원핫인코딩벡터(ohbin)으로 변환
def numbers2ohbin(numbers):

    ohbin = np.zeros(45) #45개의 빈 칸을 만듬

    for i in range(6): #여섯개의 당첨번호에 대해서 반복함
        ohbin[int(numbers[i])-1] = 1 #로또번호가 1부터 시작하지만 벡터의 인덱스 시작은 0부터 시작하므로 1을 뺌
    
    return ohbin

# 원핫인코딩벡터(ohbin)를 번호로 변환
def ohbin2numbers(ohbin):

    numbers = []
    
    for i in range(len(ohbin)):
        if ohbin[i] == 1.0: # 1.0으로 설정되어 있으면 해당 번호를 반환값에 추가한다.
            numbers.append(i+1)
    
    return numbers
    
numbers = dataframe_actual.loc[:, ['num1', 'num2', 'num3', 'num4', 'num5', 'num6']].to_numpy(int)
ohbins = list(map(numbers2ohbin, numbers))

x_samples = ohbins[0:row_count-1]
y_samples = ohbins[1:row_count]

In [92]:
tf.config.set_soft_device_placement(True)
tf.debugging.set_log_device_placement(True)

In [93]:
# 데이터 나누기
train_idx = (0, 900)
val_idx = (901, 1000)
test_idx = (1001, len(x_samples))

In [97]:
# 모델을 정의합니다.
model = keras.Sequential([
    keras.layers.LSTM(128, batch_input_shape=(1, 1, 45), return_sequences=False, stateful=True),
    keras.layers.Dense(45, activation='sigmoid')
])

# 모델을 컴파일합니다.
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 매 에포크마다 훈련과 검증의 손실 및 정확도를 기록하기 위한 변수
train_loss = []
train_acc = []
val_loss = []
val_acc = []

# 최대 100번 에포크까지 수행
for epoch in tqdm(range(200)):

    model.reset_states() # 중요! 매 에포크마다 1회부터 다시 훈련하므로 상태 초기화 필요

    batch_train_loss = []
    batch_train_acc = []
    
    for i in range(train_idx[0], train_idx[1]):
        
        xs = x_samples[i].reshape(1, 1, 45)
        ys = y_samples[i].reshape(1, 45)
        
        loss, acc = model.train_on_batch(xs, ys) #배치만큼 모델에 학습시킴

        batch_train_loss.append(loss)
        batch_train_acc.append(acc)

    train_loss.append(np.mean(batch_train_loss))
    train_acc.append(np.mean(batch_train_acc))

    batch_val_loss = []
    batch_val_acc = []

    for i in range(val_idx[0], val_idx[1]):

        xs = x_samples[i].reshape(1, 1, 45)
        ys = y_samples[i].reshape(1, 45)
        
        loss, acc = model.test_on_batch(xs, ys) #배치만큼 모델에 입력하여 나온 답을 정답과 비교함
        
        batch_val_loss.append(loss)
        batch_val_acc.append(acc)

    val_loss.append(np.mean(batch_val_loss))
    val_acc.append(np.mean(batch_val_acc))

    print('epoch {0:4d} train acc {1:0.3f} loss {2:0.3f} val acc {3:0.3f} loss {4:0.3f}'.format(epoch, np.mean(batch_train_acc), np.mean(batch_train_loss), np.mean(batch_val_acc), np.mean(batch_val_loss)))

  0%|          | 1/200 [00:06<23:11,  6.99s/it]

epoch    0 train acc 0.024 loss 0.406 val acc 0.000 loss 0.399


  1%|          | 2/200 [00:12<20:23,  6.18s/it]

epoch    1 train acc 0.020 loss 0.396 val acc 0.000 loss 0.399


  2%|▏         | 3/200 [00:18<19:26,  5.92s/it]

...

epoch  197 train acc 0.157 loss 0.006 val acc 0.000 loss 1.739


100%|█████████▉| 199/200 [19:07<00:05,  5.61s/it]

epoch  198 train acc 0.153 loss 0.006 val acc 0.000 loss 1.767


100%|██████████| 200/200 [19:12<00:00,  5.76s/it]

epoch  199 train acc 0.141 loss 0.003 val acc 0.000 loss 1.748


In [98]:
model.save("models/lstm.h5")

In [99]:
# 번호 뽑기
def gen_numbers_from_probability(nums_prob):

    ball_box = []

    for n in range(45):
        ball_count = int(nums_prob[n] * 100 + 1)
        ball = np.full((ball_count), n+1) #1부터 시작
        ball_box += list(ball)

    selected_balls = []

    while True:
        
        if len(selected_balls) == 6:
            break
        
        ball_index = np.random.randint(len(ball_box), size=1)[0]
        ball = ball_box[ball_index]

        if ball not in selected_balls:
            selected_balls.append(ball)

    return selected_balls
    
print('receive numbers')

xs = x_samples[-1].reshape(1, 1, 45)

ys_pred = model.predict_on_batch(xs)

list_numbers = []

for n in range(5):
    numbers = gen_numbers_from_probability(ys_pred[0])
    numbers.sort()
    print('{0} : {1}'.format(n, numbers))
    list_numbers.append(numbers)

receive numbers
0 : [9, 10, 12, 24, 27, 40]
1 : [9, 10, 12, 27, 40, 44]
2 : [9, 10, 12, 17, 27, 40]
3 : [5, 9, 10, 12, 40, 44]
4 : [5, 9, 12, 17, 27, 44]
